In [6]:
from keras.layers import Dropout
def build_cnn(Dropout,optimizer):
    
    # Step-1 Create Convolutional Layers
    from keras.models import Sequential
    from keras.layers import Convolution2D
    cnn=Sequential()
    cnn.add(Convolution2D(32,3,3,input_shape=(64,64,3),activation='relu'))
    
    # Step -2 Create Polling Layers
    from keras.layers import MaxPooling2D
    cnn.add(MaxPooling2D(pool_size=(2,2)))
    
    # Create Second convolution layer to get better accuracy for test set
    cnn.add(Convolution2D(32,3,3,activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2,2)))
    
    # Step-3 Create flatten to pooled layer
    from keras.layers import Flatten
    cnn.add(Flatten())
    
    # Step-4 Fully Connection Layer
    from keras.layers import Dense
    
    # First Fully connected hidden layer
    cnn.add(Dense(output_dim=128,activation='relu'))
    cnn.add(Dropout(p=Dropout))
    # Second Fully Connected hidden layer
    cnn.add(Dense(output_dim=128,activation='relu'))
    cnn.add(Dropout(p=Dropout))
    # Output layer
    cnn.add(Dense(output_dim=1,activation='sigmoid'))
    
    # Compile the CNN
    cnn.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return cnn

In [7]:
# Tunning and imporving CNN using hyper parameter combination

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
cnn=KerasClassifier(build_fn=build_cnn)
params={'optimizer':['adam','rmsprop'],
        'Dropout':[0.1,0.2,0.3,0.4],
       'batch_size':[10,20,30,40],
       'nb_epoch':[25,35,45,50]}
results=GridSearchCV(estimator=cnn,
                    param_grid=params,
                    cv=10)

In [8]:
    from keras.preprocessing.image import ImageDataGenerator
    train_datagen=ImageDataGenerator(rescale=1./255,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True)
    test_datagen=ImageDataGenerator(rescale=1./255)
    training_set=train_datagen.flow_from_directory('dataset/training_set',
                                                    # Target size should match with input_dim of convolution layer
                                                    target_size=(64,64),
                                                    batch_size=32,
                                                    class_mode='binary')
    test_set=test_datagen.flow_from_directory('dataset/test_set',
                                               target_size=(64,64),
                                               batch_size=25,
                                               class_mode='binary')
    grid_search=cnn.fit_generator(training_set,
                                      #training set data size
                                      samples_per_epoch=8000,
                                      nb_epoch=nb_epoch,
                                      validation_data=test_set,
                                      nb_val_samples=2000)
    best_parameter=grid_search.best_params_
    best_accuracy=grid_search.best_score_
    print(best_parameter)
    print(best_accuracy)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


AttributeError: 'KerasClassifier' object has no attribute 'fit_generator'